In [1]:
import json
import numpy as np
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import pickle

# Load the intents file (JSON format)
with open('intents.json') as file:
    data = json.load(file)
data

{'intents': [{'tag': 'google',
   'patterns': ['google', 'search', 'internet'],
   'responses': ['Redirecting to Google...']},
  {'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day',
    'Namaste',
    'yo'],
   'responses': ['Hello',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Get lost',
    'Till next time',
    'bbye'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I u

In [2]:
# Prepare training sentences and labels
training_sentences = []
training_labels = []
labels = []
responses = []

# Process the intents data
for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [3]:
training_sentences

['google',
 'search',
 'internet',
 'Hi there',
 'How are you',
 'Is anyone there?',
 'Hey',
 'Hola',
 'Hello',
 'Good day',
 'Namaste',
 'yo',
 'Bye',
 'See you later',
 'Goodbye',
 'Get lost',
 'Till next time',
 'bbye',
 'Thanks',
 'Thank you',
 "That's helpful",
 'Awesome, thanks',
 'Thanks for helping me',
 'How you could help me?',
 'What you can do?',
 'What help you provide?',
 'How you can be helpful?',
 'What support is offered',
 'Tell me a joke',
 'Joke',
 'Make me laugh',
 'Who are you',
 'what are you',
 'What is the time',
 'what is the date',
 'date',
 'time',
 'tell me the date',
 'day',
 'what day is is today',
 'Whats up',
 'Wazzup',
 'How are you',
 'sup',
 'How you doing',
 'haha',
 'lol',
 'rofl',
 'lmao',
 'thats funny',
 'Who made you',
 'who designed you',
 'who programmed you',
 'you are dumb',
 'shut up',
 'idiot',
 'what are you doing',
 'what are you upto',
 'Awesome',
 'Great',
 'I know',
 'ok',
 'yeah',
 'temperature',
 'weather',
 'how hot is it',
 'who 

In [4]:
training_labels

['google',
 'google',
 'google',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'goodbye',
 'goodbye',
 'goodbye',
 'goodbye',
 'goodbye',
 'goodbye',
 'thanks',
 'thanks',
 'thanks',
 'thanks',
 'thanks',
 'options',
 'options',
 'options',
 'options',
 'options',
 'jokes',
 'jokes',
 'jokes',
 'Identity',
 'Identity',
 'datetime',
 'datetime',
 'datetime',
 'datetime',
 'datetime',
 'datetime',
 'datetime',
 'whatsup',
 'whatsup',
 'whatsup',
 'whatsup',
 'whatsup',
 'haha',
 'haha',
 'haha',
 'haha',
 'haha',
 'programmer',
 'programmer',
 'programmer',
 'insult',
 'insult',
 'insult',
 'activity',
 'activity',
 'exclaim',
 'exclaim',
 'exclaim',
 'exclaim',
 'exclaim',
 'weather',
 'weather',
 'weather',
 'karan',
 'karan',
 'karan',
 'karan',
 'contact',
 'contact',
 'contact',
 'contact',
 'appreciate',
 'appreciate',
 'appreciate',
 'appreciate',
 'nicetty',
 'nicetty',
 'no',
 'no',
 'news',
 'news',
 'news'

In [5]:
labels

['google',
 'greeting',
 'goodbye',
 'thanks',
 'noanswer',
 'options',
 'jokes',
 'Identity',
 'datetime',
 'whatsup',
 'haha',
 'programmer',
 'insult',
 'activity',
 'exclaim',
 'weather',
 'karan',
 'contact',
 'appreciate',
 'nicetty',
 'no',
 'news',
 'inspire',
 'cricket',
 'song',
 'greetreply',
 'timer',
 'covid19',
 'suggest',
 'riddle',
 'age']

In [6]:
responses

[['Redirecting to Google...'],
 ['Hello', 'Good to see you again', 'Hi there, how can I help?'],
 ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
 ['Happy to help!', 'Any time!', 'My pleasure'],
 ["Sorry, can't understand you",
  'Please give me more info',
  'Not sure I understand'],
 ["I am a general purpose chatbot. My capabilities are : \n 1. I can chat with you. Try asking me for jokes or riddles! \n 2. Ask me the date and time \n 3. I can google search for you. Use format google: your query \n 4. I can get the present weather for any city. Use format weather: city name \n 5. I can get you the top 10 trending news in India. Use keywords 'Latest News' \n 6. I can get you the top 10 trending songs globally. Type 'songs' \n 7. I can set a timer for you. Enter 'set a timer: minutes to timer' \n 8. I can get the present Covid stats for any country. Use 'covid 19: world' or 'covid 19: country name' \n For suggestions to help me improve, send an email to ted.thedlbot.sugge

In [12]:
# Prepare the data for training
num_classes = len(labels)
lbl_encoder = LabelEncoder()
training_labels=lbl_encoder.fit_transform(training_labels)

# Define tokenizer and convert text data to sequences
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [14]:
word_index

{'<OOV>': 1,
 'you': 2,
 'are': 3,
 'is': 4,
 'what': 5,
 'who': 6,
 'how': 7,
 'me': 8,
 'good': 9,
 'songs': 10,
 'a': 11,
 'the': 12,
 'contact': 13,
 'day': 14,
 'time': 15,
 'thanks': 16,
 'awesome': 17,
 'date': 18,
 'i': 19,
 'karan': 20,
 'news': 21,
 'top': 22,
 'there': 23,
 'helpful': 24,
 'help': 25,
 'can': 26,
 'tell': 27,
 'joke': 28,
 'up': 29,
 'doing': 30,
 'made': 31,
 'great': 32,
 'hot': 33,
 'it': 34,
 'best': 35,
 'your': 36,
 'cricket': 37,
 'am': 38,
 "i'm": 39,
 'fine': 40,
 'useless': 41,
 'ask': 42,
 'riddle': 43,
 'google': 44,
 'search': 45,
 'internet': 46,
 'hi': 47,
 'anyone': 48,
 'hey': 49,
 'hola': 50,
 'hello': 51,
 'namaste': 52,
 'yo': 53,
 'bye': 54,
 'see': 55,
 'later': 56,
 'goodbye': 57,
 'get': 58,
 'lost': 59,
 'till': 60,
 'next': 61,
 'bbye': 62,
 'thank': 63,
 "that's": 64,
 'for': 65,
 'helping': 66,
 'could': 67,
 'do': 68,
 'provide': 69,
 'be': 70,
 'support': 71,
 'offered': 72,
 'make': 73,
 'laugh': 74,
 'today': 75,
 'whats': 76,

In [16]:
sequences

[[44],
 [45],
 [46],
 [47, 23],
 [7, 3, 2],
 [4, 48, 23],
 [49],
 [50],
 [51],
 [9, 14],
 [52],
 [53],
 [54],
 [55, 2, 56],
 [57],
 [58, 59],
 [60, 61, 15],
 [62],
 [16],
 [63, 2],
 [64, 24],
 [17, 16],
 [16, 65, 66, 8],
 [7, 2, 67, 25, 8],
 [5, 2, 26, 68],
 [5, 25, 2, 69],
 [7, 2, 26, 70, 24],
 [5, 71, 4, 72],
 [27, 8, 11, 28],
 [28],
 [73, 8, 74],
 [6, 3, 2],
 [5, 3, 2],
 [5, 4, 12, 15],
 [5, 4, 12, 18],
 [18],
 [15],
 [27, 8, 12, 18],
 [14],
 [5, 14, 4, 4, 75],
 [76, 29],
 [77],
 [7, 3, 2],
 [78],
 [7, 2, 30],
 [79],
 [80],
 [81],
 [82],
 [83, 84],
 [6, 31, 2],
 [6, 85, 2],
 [6, 86, 2],
 [2, 3, 87],
 [88, 29],
 [89],
 [5, 3, 2, 30],
 [5, 3, 2, 90],
 [17],
 [32],
 [19, 91],
 [92],
 [93],
 [94],
 [95],
 [7, 33, 4, 34],
 [6, 4, 96],
 [6, 4, 97],
 [6, 4, 20],
 [20, 98],
 [13, 99],
 [13, 20],
 [13, 100],
 [13, 101],
 [2, 3, 17],
 [2, 3, 12, 35],
 [2, 3, 32],
 [2, 3, 9],
 [34, 102, 103, 104, 105, 2],
 [9, 106],
 [107],
 [108],
 [21],
 [109, 21],
 [110, 21],
 [6, 111, 2],
 [6, 4, 36, 112],

In [18]:
padded_sequences

array([[  0,   0,   0, ...,   0,   0,  44],
       [  0,   0,   0, ...,   0,   0,  45],
       [  0,   0,   0, ...,   0,   0,  46],
       ...,
       [  0,   0,   0, ..., 127,   3,   2],
       [  0,   0,   0, ..., 129,   2,  31],
       [  0,   0,   0, ...,   4,  36, 130]])

In [20]:
# Define the model
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

# Save the model, tokenizer, and label encoder
model.save("chat_model.h5")

# Save the tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save the label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

print("Model training completed and saved!")


Epoch 1/500


C:\Users\ADMIN\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.0525 - loss: 3.4338
Epoch 2/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0504 - loss: 3.4313 
Epoch 3/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0494 - loss: 3.4291 
Epoch 4/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0348 - loss: 3.4287     
Epoch 5/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0681 - loss: 3.4254 
Epoch 6/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0619 - loss: 3.4227 
Epoch 7/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0567 - loss: 3.4246 
Epoch 8/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.0525 - loss: 3.4214  
Epoch 9/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0327 - loss: 3.4182     
Epoch 10/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0473 - loss: 3.4123 
Epoch 11/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0535 - loss: 3.4124 
Epoch 12/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0410 - loss: 3.

Model training completed and saved!


In [21]:
import json
import numpy as np
from tensorflow import keras
import pickle
import random
import colorama 
colorama.init()
from colorama import Fore, Style

# Load the intents file
with open("intents.json") as file:
    data = json.load(file)

# Function to chat with the bot
def chat():
    # Load trained model
    model = keras.models.load_model('chat_model.h5')

    # Load tokenizer and label encoder
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # Parameters
    max_len = 20

    print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()

        if inp.lower() == "quit":
            break

        # Predict the tag for the input
        result = model.predict(keras.preprocessing.sequence.pad_sequences(
            tokenizer.texts_to_sequences([inp]), truncating='post', maxlen=max_len))

        # Get the predicted tag
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        # Print the predicted tag for debugging
        print("Predicted tag:", tag)

        found_response = False
        for intent in data['intents']:
            if intent['tag'] == tag:
                response = random.choice(intent['responses'])
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL, response)
                found_response = True
                break

        # If no response is found
        if not found_response:
            print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL, "Sorry, I didn't understand that.")

# Start the chatbot
chat()


Start messaging with the bot (type quit to stop)!
User: 

 hi


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
Predicted tag: ['greeting']
ChatBot: Hi there, how can I help?
User: 

 what can you help


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Predicted tag: ['options']
ChatBot: I am a general purpose chatbot. My capabilities are : 
 1. I can chat with you. Try asking me for jokes or riddles! 
 2. Ask me the date and time 
 3. I can google search for you. Use format google: your query 
 4. I can get the present weather for any city. Use format weather: city name 
 5. I can get you the top 10 trending news in India. Use keywords 'Latest News' 
 6. I can get you the top 10 trending songs globally. Type 'songs' 
 7. I can set a timer for you. Enter 'set a timer: minutes to timer' 
 8. I can get the present Covid stats for any country. Use 'covid 19: world' or 'covid 19: country name' 
 For suggestions to help me improve, send an email to ted.thedlbot.suggestions@gmail.com . Thank you!! 
User: 

...

 okay


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
Predicted tag: ['greeting']
ChatBot: Hello
User: 

 thanks you


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
Predicted tag: ['thanks']
ChatBot: My pleasure
User: 

 quit
